In [ ]:
import sentiment_multiprocessing
from datetime import datetime
import pandas as pd
import sqlite3

### Calculation note length

In [ ]:
con_rev = sqlite3.connect('../database/review.db')
df_rev = pd.read_sql_query("SELECT * from review", con_rev)
df_rev.head()

In [ ]:
df_rev['note_length'] = df_rev.apply(lambda row: len(str(row.note).split()), axis=1)

In [ ]:
df_rev.drop(columns='index', inplace=True)
df_rev.sort_values(by='id', inplace=True)
df_rev.to_sql('review', con=con_rev, if_exists='replace')

### Sentiment analysis

In [ ]:
con_rev = sqlite3.connect('../database/review_en.db')
df_rev = pd.read_sql_query("SELECT id, note from review_en", con_rev)
df_rev.head()

In [ ]:
con_rev = sqlite3.connect('../database/review_en.db')
df_rev = pd.read_sql_query("SELECT id, note from review_en", con_rev)
results = sentiment_multiprocessing.process(df_rev)
df_results = pd.concat(results)
del df_rev
con_reviews = sqlite3.connect('../database/review_en.db')
df_reviews = pd.read_sql_query("SELECT * from review_en", con_reviews)
df_reviews.drop(columns=['index'], inplace=True)
df_final = pd.merge(df_reviews, df_results, on='id')

con_rev_en_sent = sqlite3.connect('../database/review_en_sentiment.db')
cur_rev_en_sent = con_rev_en_sent.cursor()
cur_rev_en_sent.execute('CREATE TABLE IF NOT EXISTS review_en_sentiment('
                        'id, wine_id, rating, note, created_at, likes_count, comments_count, scan_image_path, user_id, note_length, len_code, sentiment)')
df_final.to_sql('review_en_sentiment', con=con_rev_en_sent, if_exists='replace')